## Setup and Data Import

In [1]:
import sys
sys.path.insert(0, '..')

from joblib import dump, load

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn.model_selection as ms
from sklearn import preprocessing as pp
from sklearn.linear_model import \
    LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import \
    RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import recall_score

import imblearn

In [2]:
providers = load('./data/Providers_Final.pkl')
providers.set_index('Provider', inplace=True)

In [3]:
X = providers.drop('PotentialFraud', axis=1)
y = providers.PotentialFraud

## Pre-processing

In [4]:
X_train, X_test, y_train, y_test = ms.train_test_split(
    X, y, test_size = 0.3, random_state = 0, stratify=y)

# # 70/30 split gives roughly the same baseline model results
# # but saves grid_search time

In [5]:
# # Scale only the training data to avoid data leakage
scaler = pp.MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

## Modeling

In [6]:
# # Stratify folds so that classes always have the same sample ratio
skfold = ms.StratifiedKFold(n_splits=10, random_state=0, shuffle=True);

### Logistic Regression

#### Models

In [7]:
# # baseline model

# # L1 penalty for feature selection, liblinear solver faster than saga
# logRegCV = \
#     LogisticRegressionCV(penalty='l1', solver='liblinear', cv=skfold,
#                          class_weight='balanced', scoring='recall',
#                          random_state=0, n_jobs=(-1), verbose=1)

# logRegCV.fit(X_train, y_train)

# # dump(logRegCV, './data/logRegCV.pkl')

In [8]:
# # grid search with accuracy scoring metric

# logRegGSAccuracy = ms.GridSearchCV(logRegModel, param_grid=params,
#                                    cv=skfold, n_jobs=(-1), verbose=1)

# logRegAccuracy = logRegGSAccuracy.fit(X_train, y_train)
# bestLogRegAccuracy = logRegAccuracy.best_estimator_

# # dump(bestLogRegAccuracy, './data/bestLogRegAccuracy.pkl')

In [9]:
# # grid search with recall scoring metric

# # can't use scoring param, need to use recall_score()
# logRegModel = \
#     LogisticRegression(penalty='l1', solver='liblinear',
#                        class_weight='balanced', random_state=0,
#                        n_jobs=(-1), verbose=1)

# params = {'C': np.logspace(-2, 2, 50),
#           'max_iter': [100, 500, 1000]}

# logRegGS = ms.GridSearchCV(logRegModel, param_grid=params,
#                            scoring='recall', cv=skfold, verbose=1)

# logReg = logRegGS.fit(X_train, y_train)
# bestLogReg = logReg.best_estimator_

# # dump(bestLogReg, './data/bestLogReg.pkl')

#### Results

In [10]:
# # baseline model
logRegCV = load('./data/logRegCV.pkl')

print(recall_score(y_train, logRegCV.predict(X_train)))
print(recall_score(y_test, logRegCV.predict(X_test)))

0.9180790960451978
0.9144736842105263


C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator LogisticRegressionCV from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
# # grid search with accuracy scoring metric
bestLogRegAccuracy = load('./data/bestLogRegAccuracy.pkl')

# print(bestLogRegAccuracy)
print(recall_score(y_train, bestLogRegAccuracy.predict(X_train)))
print(recall_score(y_test, bestLogRegAccuracy.predict(X_test)))

0.9152542372881356
0.8289473684210527


C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
# # grid search with recall scoring metric
bestLogReg = load('./data/bestLogReg.pkl')

# print(bestLogReg)
print(recall_score(y_train, bestLogReg.predict(X_train)))
print(recall_score(y_test, bestLogReg.predict(X_test)))

0.9265536723163842
0.9210526315789473


C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
coefficients = pd.DataFrame(bestLogReg.coef_.T, index=X.columns
                           ).rename(columns = {0:'Coefficient'}
                           ).abs().sort_values(by='Coefficient',
                                               ascending=False)
coefficients[coefficients.Coefficient > 0]

,Coefficient
PatientsPerOthPhys,7.691565
Ratio_ClaimsPerPatient,3.215364
IP_Count_UniquePatients,3.138851
Perc_Outpatient,1.847467
DualPatientProvider,1.636980
OP_Perc_MultHosp,0.674412
IP_Mean_InsReimbursementRatio,0.379315
OP_Count_UniqueState,0.228130
IP_Perc_HeartFailure_Chronic,0.143150
Perc_MultHospAttPhys,0.057042


### Tree Models

#### Random Forest

In [14]:
# max_features default is 'auto' (sqrt(n_features))
randForestModel = \
    RandomForestClassifier(class_weight='balanced', random_state=0)

randForestModel.fit(X_train, y_train)

# dump(randForestModel, './data/randForestModel.pkl')

randForestModel = load('./data/randForestModel.pkl')
print(recall_score(y_train, randForestModel.predict(X_train)))
print(recall_score(y_test, randForestModel.predict(X_test)))

1.0
0.4144736842105263


C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
# params = {'n_estimators': [100,1000,5000],
#           'max_depth': np.linspace(5,50,5),
#           'max_features' : np.arange(1,11)}

# randForestGS = ms.GridSearchCV(randForestModel, param_grid=params,
#                                     scoring='recall', cv=skfold,
#                                     n_jobs=-1, verbose=1)

# randForest = randForestGS.fit(X_train, y_train)

# bestRandForest = randForest.best_estimator_
# print(bestRandForest)

best_randForest = load('./data/best_randForest.pkl')

C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\sitat\anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [16]:
# print(best_randForest)
# print(best_randForest.score(X_train, y_train))
# print(best_randForest.score(X_test, y_test))

print(recall_score(y_train, best_randForest.predict(X_train)))
print(recall_score(y_test, best_randForest.predict(X_test)))

0.9293785310734464
0.9276315789473685


#### Gradient Boosting

##### Upsampling

In [18]:
oversample = imblearn.over_sampling.SMOTE(random_state=0)
X_train_SMOTE, y_train_SMOTE = oversample.fit_resample(X_train, y_train)

In [29]:
gradBoostModel = GradientBoostingClassifier(max_features='auto', random_state=0)
gradBoostModel.fit(X_train_SMOTE, y_train_SMOTE)

print(recall_score(y_train_SMOTE, gradBoostModel.predict(X_train_SMOTE)))
print(recall_score(y_test, gradBoostModel.predict(X_test)))

0.9796096708418293
0.743421052631579


In [30]:
# dump(gradBoostModel, './data/gradBoostModel.pkl')

['./data/gradBoostModel.pkl']

In [49]:
params = {'n_estimators': [100, 1000, 5000],
          'learning_rate':[0.05, 0.1, 0.5],
          'min_samples_leaf': [1, 3, 5],
          'max_depth': np.arange(1, 20, 8),
          'max_features' : np.arange(1, 20, 8)}

gradBoostGS = ms.GridSearchCV(gradBoostModel, param_grid=params,
                                    scoring='recall', cv=skfold,
                                    n_jobs=-1, verbose=1)

gradBoost = gradBoostGS.fit(X_train_SMOTE, y_train_SMOTE)

bestGradBoost = gradBoost.best_estimator_
dump(bestGradBoost, './data/bestGradBoost.pkl')

# bestGradBoost = load('./data/bestGradBoost.pkl')
print(bestGradBoost)
print(recall_score(y_train_SMOTE, bestGradBoost.predict(X_train_SMOTE)))
print(recall_score(y_test, bestGradBoost.predict(X_test)))

Fitting 10 folds for each of 243 candidates, totalling 2430 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 25.4min
C:\Users\sitat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 176.9min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 219.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 378.4min
[Parallel(n_jobs=-1)]: Done 2430 out of 2430 | elapsed: 549.5min finished


GradientBoostingClassifier(max_depth=9, max_features=1, min_samples_leaf=3,
                           n_estimators=1000, random_state=0)


In [ ]:
def feature_importances(model):
    df = pd.DataFrame({'feature': np.array(X.columns),
                       'importance': model.feature_importances_}
                     ).sort_values('importance')
    return px.bar(df, 'importance', 'feature', height=1000)
# feature_importances(gradBoost)

In [ ]:
# X_train_reduced = pd.DataFrame(X_train, columns=X.columns)
# X_train_reduced = X_train_reduced.iloc[:,:len(gradBoost.feature_importances_
#                           [gradBoost.feature_importances_ > 0.005])];

# X_test_reduced = pd.DataFrame(X_test, columns=X.columns)
# X_test_reduced = X_test_reduced.iloc[:,:len(gradBoost.feature_importances_
#                           [gradBoost.feature_importances_ > 0.005])];

In [ ]:
# gradBoost_reduced = gradBoost.fit(X_train_reduced, y_train)

# print(np.mean(ms.cross_val_score(gradBoost_reduced, X_train_reduced, y_train, cv=10)))
# print(gradBoost_reduced.score(X_test_reduced, y_test))

# FI at all FI: 0.9406473665086488/0.9426987060998152
# FI at 0.001: 0.9429039808688451/0.9408502772643254
# FI at 0.005: 0.9279121352701092/0.9297597042513863